In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
originalDF = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/propaganda/augResults/only_arabic.csv')

In [4]:
import os
import sys 
import argparse

In [5]:
def load_NE_lists():
    ne_dict = {}
    ne_dir = '/content/gdrive/MyDrive/Colab Notebooks/propaganda/corpus-expander-master/NE/'
    for filename in os.listdir(ne_dir):
        if filename.endswith(".txt"):
            # print(filename)
            ne_file = os.path.join(ne_dir, filename)
            entities = open(ne_file).read().splitlines()
            # list.Adj.txt 
            _, entity_name, ext = filename.split('.')
            ne_dict[entity_name] = entities
            print('{} loaded. NE: {}'.format(filename, entity_name))
    return ne_dict

In [6]:
def get_more_named_entities(word, ne_dict):
    for ne_name, ne_list in ne_dict.items():
        if word in ne_list:
            return ne_list
    return None

In [7]:
def generate_sentences(NEs, words, index):
    gen_list = list()
    for ne in NEs:
        sen = list(words)
        sen[index] = ne
        gen_list.append(' '.join(sen))
    return gen_list

In [8]:
originalDF['text'] = originalDF['text'].astype("string")
originalDF['full_text'] = originalDF['full_text'].astype("string")

In [9]:
ne_dict = load_NE_lists()

ne.MascProperNames.txt loaded. NE: MascProperNames
ne.Adj.txt loaded. NE: Adj
ne.Nationalities_DefArticle.txt loaded. NE: Nationalities_DefArticle
ne.Grocery.txt loaded. NE: Grocery
ne.JobsDefArticle.txt loaded. NE: JobsDefArticle
ne.Animals.txt loaded. NE: Animals
ne.Months.txt loaded. NE: Months
ne.ColorsDefArticle.txt loaded. NE: ColorsDefArticle
ne.Jobs.txt loaded. NE: Jobs
ne.Times.txt loaded. NE: Times
ne.GroceryDefArticle.txt loaded. NE: GroceryDefArticle
ne.Ordinals.txt loaded. NE: Ordinals
ne.Countries.txt loaded. NE: Countries
ne.Nationalities.txt loaded. NE: Nationalities
ne.Meetings.txt loaded. NE: Meetings
ne.Season.txt loaded. NE: Season
ne.Nationalities_DefArticlePlular.txt loaded. NE: Nationalities_DefArticlePlular
ne.Colors.txt loaded. NE: Colors
ne.FemProberNames.txt loaded. NE: FemProberNames
ne.Places.txt loaded. NE: Places
ne.JobsPlur.txt loaded. NE: JobsPlur
ne.Currency.txt loaded. NE: Currency
ne.Days.txt loaded. NE: Days
ne.Transportation.txt loaded. NE: Transpo

In [10]:
originalDF = originalDF.drop(originalDF[(originalDF['text'].isnull()) & (originalDF['technique'] != 'no technique')].index)

In [11]:
originalDF[(originalDF['text'].isnull()) & (originalDF['technique'] != 'no technique')]

,Unnamed: 0,start,end,technique,text,full_text,id,start_new,end_new


In [12]:
unique_list = originalDF.technique.unique()
unique_list

array(['no technique', 'Exaggeration/Minimisation', 'Smears',
       'Loaded Language', 'Appeal to fear/prejudice',
       'Name calling/Labeling', 'Slogans', 'Repetition', 'Doubt',
       'Obfuscation, Intentional vagueness, Confusion', 'Flag-waving',
       'Glittering generalities (Virtue)',
       'Presenting Irrelevant Data (Red Herring)', 'Appeal to authority',
       'Whataboutism', 'Black-and-white Fallacy/Dictatorship',
       'Thought-terminating cliché', 'Causal Oversimplification'],
      dtype=object)

In [13]:
from tqdm import tqdm
import numpy as np

def augment_text(df,tech):

      row_list=[]
      ##selecting the minority class samples
      df_n=df[df.technique==tech].reset_index(drop=True)
      

      for j in tqdm(np.random.randint(0,len(df_n),100)):
        
        text = df_n.iloc[j]['text'] 
        full_text = df_n.iloc[j]['full_text']
        technique = df_n.iloc[j]['technique']
        start = df_n.iloc[j]['start']
        end = df_n.iloc[j]['end']
        id = df_n.iloc[j]['id']
        start_new = df_n.iloc[j]['start_new'] 
        end_new = df_n.iloc[j]['end_new']

        if(technique != 'no technique' ):
          
          words = text.strip().split()
          for i, word in enumerate(words):
            NEs = get_more_named_entities(word, ne_dict)
            if NEs:
              new_sentns = generate_sentences(NEs, words, i)
              for x in range(len(new_sentns)):
                one_row = dict()
                augmented_text = new_sentns[x]
                
                augmented_text = ''.join(augmented_text)
                #augmented_text = augmented_text.strip()
                augmented_full_text = full_text.replace(text, augmented_text)
                one_row['full_text'] = full_text
                one_row['start'] = start
                one_row['end'] = end
                one_row['technique'] = technique
                one_row['text'] = text
                one_row['id'] = id
                one_row['start_new'] = start_new
                one_row['end_new'] = end_new  
                one_row['augmented_text'] = augmented_text
                one_row['augmented_full_text'] = augmented_full_text

                row_list.append(one_row)

        else:
          
          words = full_text.strip().split()
          for i, word in enumerate(words):
            NEs = get_more_named_entities(word, ne_dict)
            if NEs:
              new_sentns = generate_sentences(NEs, words, i)
              for x in range(len(new_sentns)):
                one_row = dict()
                augmented_full_text = new_sentns[x]
                
                augmented_full_text = ''.join(augmented_full_text)
                #augmented_text = augmented_text.strip()
                augmented_text = ''
                one_row['full_text'] = full_text
                one_row['start'] = start
                one_row['end'] = end
                one_row['technique'] = technique
                one_row['text'] = text
                one_row['id'] = id
                one_row['start_new'] = start_new
                one_row['end_new'] = end_new  
                one_row['augmented_text'] = augmented_text
                one_row['augmented_full_text'] = augmented_full_text

                row_list.append(one_row)      
        
      df=pd.DataFrame(row_list)
      return df

In [14]:
augmented_df = pd.DataFrame()

for i in range(len(unique_list)):
    temp_aug_df = pd.DataFrame()
    temp_aug_df = augment_text(originalDF,unique_list[i])
    augmented_df = augmented_df.append(temp_aug_df)

print('There are {} rows and {} columns in train'.format(augmented_df.shape[0],augmented_df.shape[1]))    
bool_series = augmented_df.duplicated(subset=['augmented_full_text','technique'], keep = 'first')
print('Boolean series:')
print(bool_series)
print('\n')
print('DataFrame after removing duplicates found in the Name column:')
CorpsExpand = augmented_df[~bool_series]
print('There are {} rows and {} columns in train'.format(CorpsExpand.shape[0],CorpsExpand.shape[1])) 

print(CorpsExpand['technique'].value_counts())
path = '/content/gdrive/MyDrive/Colab Notebooks/propaganda/augResults/CorpsExpand.csv'
with open(path, 'w', encoding = 'utf-8') as f:
  CorpsExpand.to_csv(f)

100%|██████████| 100/100 [00:00<00:00, 196.68it/s]


There are 274296 rows and 10 columns in train
Boolean series:
0       False
1       False
2       False
3       False
4       False
        ...  
5218     True
5219     True
5220     True
5221     True
5222     True
Length: 274296, dtype: bool


DataFrame after removing duplicates found in the Name column:
There are 88878 rows and 10 columns in train
no technique                                     34175
Smears                                           16273
Glittering generalities (Virtue)                 10924
Appeal to authority                               8016
Appeal to fear/prejudice                          4992
Name calling/Labeling                             4187
Doubt                                             3751
Exaggeration/Minimisation                         1801
Flag-waving                                       1607
Loaded Language                                   1604
Whataboutism                                       998
Causal Oversimplification                 

In [15]:
augmented_df

,full_text,start,end,technique,text,id,start_new,end_new,augmented_text,augmented_full_text
0,الوصال منذر من فريق أرض اللبان ما المانع أن يك...,0,0,no technique,<NA>,1373965057298694148,0.0,0.0,,الوصال ﻿أحسان من فريق أرض اللبان ما المانع أن ...
1,الوصال منذر من فريق أرض اللبان ما المانع أن يك...,0,0,no technique,<NA>,1373965057298694148,0.0,0.0,,الوصال ابو بكر من فريق أرض اللبان ما المانع أن...
2,الوصال منذر من فريق أرض اللبان ما المانع أن يك...,0,0,no technique,<NA>,1373965057298694148,0.0,0.0,,الوصال ابو مازن من فريق أرض اللبان ما المانع أ...
3,الوصال منذر من فريق أرض اللبان ما المانع أن يك...,0,0,no technique,<NA>,1373965057298694148,0.0,0.0,,الوصال ابو عمار من فريق أرض اللبان ما المانع أ...
4,الوصال منذر من فريق أرض اللبان ما المانع أن يك...,0,0,no technique,<NA>,1373965057298694148,0.0,0.0,,الوصال أحمد من فريق أرض اللبان ما المانع أن يك...
...,...,...,...,...,...,...,...,...,...,...
7387,بين سام وعمار حروب ونزاعات تنشأ حتى اليوم بسبب...,15,76,Causal Oversimplification,حروب ونزاعات تنشأ حتى اليوم بسبب انكار تركيا ل...,1389489981031206912,14.0,75.0,حروب ونزاعات تنشأ حتى اليوم بسبب انكار اميركا ...,بين سام وعمار حروب ونزاعات تنشأ حتى اليوم بسبب...
7388,بين سام وعمار حروب ونزاعات تنشأ حتى اليوم بسبب...,15,76,Causal Oversimplification,حروب ونزاعات تنشأ حتى اليوم بسبب انكار تركيا ل...,1389489981031206912,14.0,75.0,حروب ونزاعات تنشأ حتى اليوم بسبب انكار أميركا ...,بين سام وعمار حروب ونزاعات تنشأ حتى اليوم بسبب...
7389,بين سام وعمار حروب ونزاعات تنشأ حتى اليوم بسبب...,15,76,Causal Oversimplification,حروب ونزاعات تنشأ حتى اليوم بسبب انكار تركيا ل...,1389489981031206912,14.0,75.0,حروب ونزاعات تنشأ حتى اليوم بسبب انكار اليابان...,بين سام وعمار حروب ونزاعات تنشأ حتى اليوم بسبب...
7390,بين سام وعمار حروب ونزاعات تنشأ حتى اليوم بسبب...,15,76,Causal Oversimplification,حروب ونزاعات تنشأ حتى اليوم بسبب انكار تركيا ل...,1389489981031206912,14.0,75.0,حروب ونزاعات تنشأ حتى اليوم بسبب انكار طاجكستا...,بين سام وعمار حروب ونزاعات تنشأ حتى اليوم بسبب...


In [15]:
CorpsExpand[CorpsExpand.technique=='Presenting Irrelevant Data (Red Herring)']

,full_text,start,end,technique,text,id,start_new,end_new,augmented_text,augmented_full_text
0,شاهد شابة سعودية تقع في كارثة وتدهس جمعا من ال...,5,66,Presenting Irrelevant Data (Red Herring),شابة سعودية تقع في كارثة وتدهس جمعا من الناس ب...,1367143382750003203,5.0,66.0,شابة ﻿اردنية تقع في كارثة وتدهس جمعا من الناس ...,شاهد شابة ﻿اردنية تقع في كارثة وتدهس جمعا من ا...
1,شاهد شابة سعودية تقع في كارثة وتدهس جمعا من ال...,5,66,Presenting Irrelevant Data (Red Herring),شابة سعودية تقع في كارثة وتدهس جمعا من الناس ب...,1367143382750003203,5.0,66.0,شابة أردنية تقع في كارثة وتدهس جمعا من الناس ب...,شاهد شابة أردنية تقع في كارثة وتدهس جمعا من ال...
2,شاهد شابة سعودية تقع في كارثة وتدهس جمعا من ال...,5,66,Presenting Irrelevant Data (Red Herring),شابة سعودية تقع في كارثة وتدهس جمعا من الناس ب...,1367143382750003203,5.0,66.0,شابة مصرية تقع في كارثة وتدهس جمعا من الناس بس...,شاهد شابة مصرية تقع في كارثة وتدهس جمعا من الن...
3,شاهد شابة سعودية تقع في كارثة وتدهس جمعا من ال...,5,66,Presenting Irrelevant Data (Red Herring),شابة سعودية تقع في كارثة وتدهس جمعا من الناس ب...,1367143382750003203,5.0,66.0,شابة فلسطينية تقع في كارثة وتدهس جمعا من الناس...,شاهد شابة فلسطينية تقع في كارثة وتدهس جمعا من ...
4,شاهد شابة سعودية تقع في كارثة وتدهس جمعا من ال...,5,66,Presenting Irrelevant Data (Red Herring),شابة سعودية تقع في كارثة وتدهس جمعا من الناس ب...,1367143382750003203,5.0,66.0,شابة سعودية تقع في كارثة وتدهس جمعا من الناس ب...,شاهد شابة سعودية تقع في كارثة وتدهس جمعا من ال...
5,شاهد شابة سعودية تقع في كارثة وتدهس جمعا من ال...,5,66,Presenting Irrelevant Data (Red Herring),شابة سعودية تقع في كارثة وتدهس جمعا من الناس ب...,1367143382750003203,5.0,66.0,شابة مغربية تقع في كارثة وتدهس جمعا من الناس ب...,شاهد شابة مغربية تقع في كارثة وتدهس جمعا من ال...
6,شاهد شابة سعودية تقع في كارثة وتدهس جمعا من ال...,5,66,Presenting Irrelevant Data (Red Herring),شابة سعودية تقع في كارثة وتدهس جمعا من الناس ب...,1367143382750003203,5.0,66.0,شابة سورية تقع في كارثة وتدهس جمعا من الناس بس...,شاهد شابة سورية تقع في كارثة وتدهس جمعا من الن...
7,شاهد شابة سعودية تقع في كارثة وتدهس جمعا من ال...,5,66,Presenting Irrelevant Data (Red Herring),شابة سعودية تقع في كارثة وتدهس جمعا من الناس ب...,1367143382750003203,5.0,66.0,شابة عراقية تقع في كارثة وتدهس جمعا من الناس ...,شاهد شابة عراقية تقع في كارثة وتدهس جمعا من ا...
8,شاهد شابة سعودية تقع في كارثة وتدهس جمعا من ال...,5,66,Presenting Irrelevant Data (Red Herring),شابة سعودية تقع في كارثة وتدهس جمعا من الناس ب...,1367143382750003203,5.0,66.0,شابة عمانية تقع في كارثة وتدهس جمعا من الناس ...,شاهد شابة عمانية تقع في كارثة وتدهس جمعا من ا...
9,شاهد شابة سعودية تقع في كارثة وتدهس جمعا من ال...,5,66,Presenting Irrelevant Data (Red Herring),شابة سعودية تقع في كارثة وتدهس جمعا من الناس ب...,1367143382750003203,5.0,66.0,شابة سودانية تقع في كارثة وتدهس جمعا من الناس ...,شاهد شابة سودانية تقع في كارثة وتدهس جمعا من ا...


In [45]:
row_list=[]
##selecting the minority class samples
df_n=originalDF[originalDF.technique=='Slogans'].reset_index(drop=True)


for j in tqdm(np.random.randint(0,len(df_n),10)):
    
    text = df_n.iloc[j]['text'] 
    full_text = df_n.iloc[j]['full_text']
    technique = df_n.iloc[j]['technique']
    start = df_n.iloc[j]['start']
    end = df_n.iloc[j]['end']
    id = df_n.iloc[j]['id']
    start_new = df_n.iloc[j]['start_new'] 
    end_new = df_n.iloc[j]['end_new']

    if(technique != 'no technique' ):
      
      words = text.strip().split()
     
      for i, word in enumerate(words):
        NEs = get_more_named_entities(word, ne_dict)
        
        if NEs:
          new_sentns = generate_sentences(NEs, words, i)
          for x in range(len(new_sentns)):
            one_row = dict()
            augmented_text = new_sentns[x]
            
            augmented_text = ''.join(augmented_text)
            #augmented_text = augmented_text.strip()
            augmented_full_text = full_text.replace(text, augmented_text)
            one_row['full_text'] = full_text
            one_row['start'] = start
            one_row['end'] = end
            one_row['technique'] = technique
            one_row['text'] = text
            one_row['id'] = id
            one_row['start_new'] = start_new
            one_row['end_new'] = end_new  
            one_row['augmented_text'] = augmented_text
            one_row['augmented_full_text'] = augmented_full_text

            row_list.append(one_row)

df=pd.DataFrame(row_list)
df

100%|██████████| 10/10 [00:00<00:00, 642.30it/s]


,full_text,start,end,technique,text,id,start_new,end_new,augmented_text,augmented_full_text
0,انزلقت أول مركبة فضائية صينية على المريخ من من...,0,92,"Obfuscation, Intentional vagueness, Confusion",انزلقت أول مركبة فضائية صينية على المريخ من من...,7063,0.0,92.0,انزلقت أول مركبة فضائية صينية على المريخ من من...,انزلقت أول مركبة فضائية صينية على المريخ من من...
1,انزلقت أول مركبة فضائية صينية على المريخ من من...,0,92,"Obfuscation, Intentional vagueness, Confusion",انزلقت أول مركبة فضائية صينية على المريخ من من...,7063,0.0,92.0,انزلقت أول مركبة فضائية صينية على المريخ من من...,انزلقت أول مركبة فضائية صينية على المريخ من من...
2,انزلقت أول مركبة فضائية صينية على المريخ من من...,0,92,"Obfuscation, Intentional vagueness, Confusion",انزلقت أول مركبة فضائية صينية على المريخ من من...,7063,0.0,92.0,انزلقت أول مركبة فضائية صينية على المريخ من من...,انزلقت أول مركبة فضائية صينية على المريخ من من...
3,انزلقت أول مركبة فضائية صينية على المريخ من من...,0,92,"Obfuscation, Intentional vagueness, Confusion",انزلقت أول مركبة فضائية صينية على المريخ من من...,7063,0.0,92.0,انزلقت أول مركبة فضائية صينية على المريخ من من...,انزلقت أول مركبة فضائية صينية على المريخ من من...
4,انزلقت أول مركبة فضائية صينية على المريخ من من...,0,92,"Obfuscation, Intentional vagueness, Confusion",انزلقت أول مركبة فضائية صينية على المريخ من من...,7063,0.0,92.0,انزلقت أول مركبة فضائية صينية على المريخ من من...,انزلقت أول مركبة فضائية صينية على المريخ من من...
5,انزلقت أول مركبة فضائية صينية على المريخ من من...,0,92,"Obfuscation, Intentional vagueness, Confusion",انزلقت أول مركبة فضائية صينية على المريخ من من...,7063,0.0,92.0,انزلقت أول مركبة فضائية صينية على المريخ من من...,انزلقت أول مركبة فضائية صينية على المريخ من من...
6,انزلقت أول مركبة فضائية صينية على المريخ من من...,0,92,"Obfuscation, Intentional vagueness, Confusion",انزلقت أول مركبة فضائية صينية على المريخ من من...,7063,0.0,92.0,انزلقت أول مركبة فضائية صينية على المريخ من من...,انزلقت أول مركبة فضائية صينية على المريخ من من...
7,انزلقت أول مركبة فضائية صينية على المريخ من من...,0,92,"Obfuscation, Intentional vagueness, Confusion",انزلقت أول مركبة فضائية صينية على المريخ من من...,7063,0.0,92.0,انزلقت أول مركبة فضائية صينية على المريخ من من...,انزلقت أول مركبة فضائية صينية على المريخ من من...
8,انزلقت أول مركبة فضائية صينية على المريخ من من...,0,92,"Obfuscation, Intentional vagueness, Confusion",انزلقت أول مركبة فضائية صينية على المريخ من من...,7063,0.0,92.0,انزلقت أول مركبة فضائية صينية على المريخ من من...,انزلقت أول مركبة فضائية صينية على المريخ من من...
9,انزلقت أول مركبة فضائية صينية على المريخ من من...,0,92,"Obfuscation, Intentional vagueness, Confusion",انزلقت أول مركبة فضائية صينية على المريخ من من...,7063,0.0,92.0,انزلقت أول مركبة فضائية صينية على المريخ من من...,انزلقت أول مركبة فضائية صينية على المريخ من من...


In [39]:
df

""


In [28]:
augmented_df = pd.DataFrame()
temp_aug_df = pd.DataFrame()
temp_aug_df = augment_text(originalDF,unique_list[2])
augmented_df = augmented_df.append(temp_aug_df)

100%|██████████| 10/10 [00:00<00:00, 164.44it/s]


In [29]:
augmented_df

,full_text,start,end,technique,text,id,start_new,end_new,augmented_text,augmented_full_text
0,بشار الأسد يدنس دوما لينتخب نفسه,0,36,Smears,بشار الأسد يدنس دوما لينتخب نفسه,7290,0.0,32.0,﻿أحسان الأسد يدنس دوما لينتخب نفسه,﻿أحسان الأسد يدنس دوما لينتخب نفسه
1,بشار الأسد يدنس دوما لينتخب نفسه,0,36,Smears,بشار الأسد يدنس دوما لينتخب نفسه,7290,0.0,32.0,ابو بكر الأسد يدنس دوما لينتخب نفسه,ابو بكر الأسد يدنس دوما لينتخب نفسه
2,بشار الأسد يدنس دوما لينتخب نفسه,0,36,Smears,بشار الأسد يدنس دوما لينتخب نفسه,7290,0.0,32.0,ابو مازن الأسد يدنس دوما لينتخب نفسه,ابو مازن الأسد يدنس دوما لينتخب نفسه
3,بشار الأسد يدنس دوما لينتخب نفسه,0,36,Smears,بشار الأسد يدنس دوما لينتخب نفسه,7290,0.0,32.0,ابو عمار الأسد يدنس دوما لينتخب نفسه,ابو عمار الأسد يدنس دوما لينتخب نفسه
4,بشار الأسد يدنس دوما لينتخب نفسه,0,36,Smears,بشار الأسد يدنس دوما لينتخب نفسه,7290,0.0,32.0,أحمد الأسد يدنس دوما لينتخب نفسه,أحمد الأسد يدنس دوما لينتخب نفسه
...,...,...,...,...,...,...,...,...,...,...
4879,عقوبات إدارة ترامب على المحكمة الجنائية الدولي...,0,118,Smears,عقوبات إدارة ترامب على المحكمة الجنائية الدولي...,1301536953708544001,0.0,115.0,عقوبات إدارة ترامب على المحكمة الجنائية الدولي...,عقوبات إدارة ترامب على المحكمة الجنائية الدولي...
4880,عقوبات إدارة ترامب على المحكمة الجنائية الدولي...,0,118,Smears,عقوبات إدارة ترامب على المحكمة الجنائية الدولي...,1301536953708544001,0.0,115.0,عقوبات إدارة ترامب على المحكمة الجنائية الدولي...,عقوبات إدارة ترامب على المحكمة الجنائية الدولي...
4881,عقوبات إدارة ترامب على المحكمة الجنائية الدولي...,0,118,Smears,عقوبات إدارة ترامب على المحكمة الجنائية الدولي...,1301536953708544001,0.0,115.0,عقوبات إدارة ترامب على المحكمة الجنائية الدولي...,عقوبات إدارة ترامب على المحكمة الجنائية الدولي...
4882,عقوبات إدارة ترامب على المحكمة الجنائية الدولي...,0,118,Smears,عقوبات إدارة ترامب على المحكمة الجنائية الدولي...,1301536953708544001,0.0,115.0,عقوبات إدارة ترامب على المحكمة الجنائية الدولي...,عقوبات إدارة ترامب على المحكمة الجنائية الدولي...


In [ ]:
augmented_df